In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from utils import *
from copy import copy

# dataSetPath = '../../../dataset/PHEME/'
# dim = 5000

'''
threadIds 事件ID集合
postIds 帖子ID集合
rumorTags 事件对应的标签
structures 事件包含的帖子及相应的回帖结构
posts 每一个帖子的发布时间和内容
users 每一个帖子对应的时间和用户信息
label2Index 标签与谣言类型对应关系
"label2Index": {
		"non-rumor": 0,
		"true": 1,
		"false": 2,
		"unverified": 3
	}
'''

with open('./trainSet.json', 'r') as f:
    content = f.read()
trainSet = json.loads(content)
#帖子内容集合(按照事件及其事件顺序排序)
threads = [] #[[事件所包含的帖子文本集合（按照时间排序）],...]
threadUsers = []
# print(len(trainSet['threadIds']))
for threadId in trainSet['threadIds']:
    thread = []
    threadUser = []
    #事件ID
    structure = trainSet['structures'][threadId]
    #事件中所包含的帖子的ID集合
    ids = flattenStructure(structure)
    #发帖时间--->帖子ID
    time2Id = {}
    time2IdUsers = {}
    for id in ids:
        if id in trainSet['posts']:
            time2Id[str(trainSet['posts'][id]['time'])] = id
            time2IdUsers[str(trainSet['users'][id]['created_at'])] = id
    # post按照时间先后排序
    time2Id = sorted(time2Id.items(), key=lambda d: d[0])
    time2IdUsers = sorted(time2IdUsers.items(), key=lambda d: d[0])
    for (time, id) in time2Id:
        if id in trainSet['posts']:
            #帖子的内容
            thread.append(trainSet['posts'][id]['text'])
    for (time, id) in time2IdUsers:
        if id in trainSet['users']:
            #帖子的内容
            threadUser.append(trainSet['users'][id]['user'])
    threads.append(thread)
    threadUsers.append(threadUser)

with open('./testSet.json', 'r') as f:
    content = f.read()
testSet = json.loads(content)
# print(len(testSet['threadIds']))
for threadId in testSet['threadIds']:
    thread = []
    threadUser = []
    structure = testSet['structures'][threadId]
    ids = flattenStructure(structure)
    time2Id = {}
    time2IdUsers = {}
    for id in ids:
        if id in testSet['posts']:
            time2Id[str(testSet['posts'][id]['time'])] = id
            time2IdUsers[str(testSet['users'][id]['created_at'])] = id
    # post按照时间先后排序
    time2Id = sorted(time2Id.items(), key=lambda d: d[0])
    time2IdUsers = sorted(time2IdUsers.items(), key=lambda d: d[0])
    for (time, id) in time2Id:
        if id in testSet['posts']:
            thread.append(testSet['posts'][id]['text'])
    for (time, id) in time2IdUsers:
        if id in trainSet['users']:
            #帖子的内容
            threadUser.append(testSet['users'][id]['user'])
    threads.append(thread)
    threadUsers.append(threadUser)

cropus = []
for thread in threads:
    # print(f"thread,{thread}")
    for text in thread:
        # print(f"text:{text}")
        cropus.append(text)
# 存储的事每一个回复的文本[[],[],[]...]长度101228 代表所有的回复的数量有101228个（包括原始推文）
print("cropus:",len(cropus))

# https://huaweicloud.csdn.net/638069c2dacf622b8df87387.html?spm=1001.2101.3001.6650.1&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Eactivity-1-113820065-blog-123371873.235%5Ev30%5Epc_relevant_default_base3&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Eactivity-1-113820065-blog-123371873.235%5Ev30%5Epc_relevant_default_base3&utm_relevant_index=2
#https://zhuanlan.zhihu.com/p/67883024
# tfidf_vec = TfidfVectorizer(max_features=5000)#字典大小是5000 即如果进行向量嵌入则，每一个回复的文本都是一个5000维的tf-idf向量
# tfidf_vec.fit(cropus)#只是建立了词典  没有对原始文本进行向量嵌入
# print(f"vocal:{len(tfidf_vec.vocabulary_)}")
label2IndexRumor = trainSet['label2Index'] if 'label2Index' in trainSet else None
# print(len(tfidf_vec.vocabulary_))

cropus: 101228


In [2]:
with open('./testSet.json', 'r') as f:
    content = f.read()
testSet = json.loads(content)
totalSet = {}
for key in trainSet:
    if isinstance(trainSet[key], list):
        totalSet[key] = copy(trainSet[key])
        totalSet[key] += testSet[key]
    elif isinstance(trainSet[key], dict):
        totalSet[key] = copy(trainSet[key])
        totalSet[key].update(testSet[key])
print(totalSet.keys())

dict_keys(['threadIds', 'postIds', 'rumorTags', 'structures', 'posts', 'users', 'label2Index'])


In [3]:
with open('./PHEME_id_label.txt', 'w') as f:
    for key in totalSet['rumorTags']:
        f.write('{:s} {:d}\n'.format(key, totalSet['rumorTags'][key]))

In [4]:
def extract_usr_attributes(user_dict, source_post_time,reply_post_time):
    # 拿到用户字典
    # user_dict = post_dict['user']
    user_dict = user_dict
    # 本篇回复的创建时间
    # reply_post_time = time_stamp(post_dict['created_at'])
    # Extract User Attributes from user dictionary.
    rep = []
    vec = ""
    # 1.是否使用
    vec += '{:d} '.format(1 if user_dict['profile_use_background_image'] else 0)
    # rep.append(1 if user_dict['profile_use_background_image'] else 0)
    # 2.是否是认证账号
    vec += '{:d} '.format(1 if user_dict['verified'] else 0)
    # rep.append(1 if user_dict['verified'] else 0)
    # 3.粉丝数
    vec += '{:d} '.format(user_dict['followers_count'])
    # rep.append(user_dict['followers_count'])
    # 4.关注数
    vec += '{:d} '.format(user_dict['listed_count'])
    # rep.append(user_dict['listed_count'])
    # 5.
    vec += '{:d} '.format(user_dict['statuses_count'])
    # rep.append(user_dict['statuses_count'])
    # 6. Holy some users have no descriptions (NoneType) 自我描述的长度
    if user_dict['description'] is not None:
        #todo 这些地方是直接计算len  还是按照单词个数计算len比较好？
        vec += '{:d} '.format(len(user_dict['description']))
        # rep.append(len(user_dict['description']))
    else:
        vec += '{:d} '.format(0)
        # rep.append(0)
    # 7.
    vec += '{:d} '.format(user_dict['friends_count'])
    # rep.append(user_dict['friends_count'])
    # 8.定位是否开启
    vec += '{:d} '.format(1 if user_dict['geo_enabled'] else 0)
    # rep.append(1 if user_dict['geo_enabled'] else 0)
    # 9.
    vec += '{:d} '.format(1 if user_dict['profile_background_tile'] else 0)
    # rep.append(1 if user_dict['profile_background_tile'] else 0)
    # 10.
    vec += '{:d} '.format(user_dict['favourites_count'])
    # rep.append(user_dict['favourites_count'])
    # 11.
    vec += '{:d} '.format(1 if user_dict['contributors_enabled'] else 0)
    # rep.append(1 if user_dict['contributors_enabled'] else 0)
    # 12. Reply time over the source post
    vec += '{:f}\t'.format(reply_post_time - source_post_time)
    # rep.append(reply_post_time - source_post_time)
    return vec

def structure2graph(glove, structure,source_post_time, posts,users, contents, id, curIndex: list, parent=None):
    if id in posts:
        post = posts[id]['text']
        user_dict = users[id]['user']
        time = users[id]['created_at']
        word2feq = {}
        # for word in post.split(' '):
        #     if word in word2feq:
        #         word2feq[word] += 1
        #     else:
        #         word2feq[word] = 1



        content = ''
        if parent == None:
            #id     None    curIndex[0]
            content += '{:s}\t{:s}\t{:d}\t'.format(id, 'None', curIndex[0])
        else:
            #id     parent    curIndex[0]
            content += '{:s}\t{:d}\t{:d}\t'.format(id, parent, curIndex[0])

        user_vec = extract_usr_attributes(user_dict,source_post_time,time)
        content += user_vec
        vec = ''
        #glove 词表中的index
        for word in post.split(' '):
            if word.lower() in glove.stoi:
                vec += '{:d} '.format(glove.stoi[word.lower()])
        # for word in word2feq:
        #     if word in wordlist :
        #         vec += '{:d}:{:d} '.format(wordlist[word], word2feq[word])
        vec = vec[0:-1] + '\n'
        content += vec
        if len(vec) == 1:
            print(post)
        else:
            contents.append(content)

    if not structure:
        return

    p = cur[0]
    for cid in structure:
        cur[0] += 1
        structure2graph(glove, structure[cid],source_post_time, posts,users, contents, cid, curIndex, p)

    return

In [5]:
from config import glove
with open('./PHEMEtree.txt', 'w') as f:
    for tid in totalSet['threadIds']:
        struct = totalSet['structures'][tid]
        posts = totalSet['posts']
        source_post_time = posts[tid]['time']
        users = totalSet['users']
        contents = []
        cur = [1]
        structure2graph(glove, struct, source_post_time,posts,users, contents, tid, cur, parent=None)
        for content in contents:
            f.write(content)